# Subpockets vs. conformations

Check if our `kissim` subpockets are reasonable across different kinase conformations.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import pandas as pd

from opencadd.databases.klifs import setup_remote
from opencadd.databases.klifs.utils import metadata_to_filepath
from opencadd.structure.pocket import KlifsPocket, PocketViewer

## Data retrieval (KLIFS structures)

In [ ]:
session = setup_remote()

In [ ]:
structures = session.structures.all_structures()

Define a function that allows us to get _X_ random structures.

In [ ]:
def random_pockets(structures, n_structures, seed, anchor_residues, subpocket_names, subpocket_colors, klifs_session):
    
    random_structures = structures.sample(n=n_structures, random_state=seed)
    subpockets = {
        "anchor_residue.klifs_ids": anchor_residues,
        "subpocket.name": subpocket_names,
        "subpocket.color": subpocket_colors
    }
    pockets = []
    for structure_klifs_id in random_structures["structure.klifs_id"]:
        print(structure_klifs_id)
        pocket = KlifsPocket.from_structure_klifs_id(structure_klifs_id, subpockets, klifs_session=klifs_session)
        pockets.append(pocket)
        
    return pockets

In [ ]:
N_STRUCTURES = 3
SEED = 1

## Anchor residue selection

In [ ]:
subpocket_names = [
    "hinge_region", "hinge_region_kinfraglib_ap", 
    "dfg_region", "dfg_region_kinfraglib_b2", 
    "front_pocket", "front_pocket2", "front_pocket_kinfraglib_fp"
]
anchor_residues = [
    [16, 47, 80], [15, 46, 51, 75],
    [19, 24, 81], [18, 24, 70, 83], 
    [6, 48, 75], [10, 48, 75], [10, 51, 72, 81]
]

In [ ]:
pockets = random_pockets(
    structures, 
    N_STRUCTURES, 
    SEED, 
    anchor_residues, 
    subpocket_names, 
    ["magenta", "violet", "cyan", "darkturquoise", "green", "lime", "lightgreen"], 
    session
)

In [ ]:
viewer = PocketViewer()
for pocket in pockets:
    viewer.add_pocket(pocket, sphere_opacity=0.7)
viewer.viewer

## Subpocket robustness across kinase conformations [WIP]

In [ ]:
SUBPOCKET_NAMES = ["hinge_region", "dfg_region", "front_pocket"]
ANCHOR_RESIDUES = [[16, 47, 80], [19, 24, 81], [6, 48, 75]]

### DFG conformations

In [ ]:
pockets_by_conformation = {
    "in": random_pockets(
        structures[(structures["structure.dfg"] == "in")], 
        N_STRUCTURES, 
        SEED, 
        ANCHOR_RESIDUES, 
        SUBPOCKET_NAMES, 
        ["magenta", "cyan", "green"], 
        session),
    "out": random_pockets(
        structures[(structures["structure.dfg"] == "out")], 
        N_STRUCTURES, 
        SEED, 
        ANCHOR_RESIDUES, 
        SUBPOCKET_NAMES, 
        ["violet", "turquoise", "lime"], 
        session
    ),
}

In [ ]:
viewer = PocketViewer()
for _, pockets in pockets_by_conformation.items():
    for pocket in pockets:
        viewer.add_pocket(pocket, sphere_opacity=1)
viewer.viewer

In [ ]:
viewer.structure_names

In [ ]:
viewer.show_only(5730)

### aC-helix conformation [WIP]

__KLIFS definitions__

* aC helix conformation: Distance between the Ca atoms of residue 82 (DFG) and conserved residue 24 (aC helix).
  * aC-in: $[4, 7.2]$
  * aC-out: $>9.3$
  * aC-out-like: $[7.2, 9.3]$
* DFG conformation: Manually.

__Examples by Ung _et al.___

BRAF structures in different conformations taken from [Ung _et al._ (2018)](https://www.sciencedirect.com/science/article/pii/S2451945618301491).

|             | aC-in        | aC-out       |
|-------------|--------------|--------------|
| __DFG-in__  | 2FB8 _CIDI_  | 4EHG _CODI_  |
| __DFG-out__ | 1UWH _CIDO_  | 4XV9 _CODO_  |

_Note_: KLIFS conformation annotation differs from Ung _et al._.